<a href="https://colab.research.google.com/github/aravs1/Foodie-Finder/blob/main/Yelp_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import requests
import pandas as pd
import time

# API Key
API_KEY = "6ca95wYJMQGrfd-COZR71UxqB7BhxuCSOaurylwvdKjcljEU9hxosKUCx0-0_TTvFNxqmLscQnFSDECaTUXP1YS_bvo8bbU_kQTkzb-_QK4RyjX9DLHAc6fPD8cKZ3Yx"

# API Endpoint and Headers
url_business = "https://api.yelp.com/v3/businesses/search"
url_reviews = "https://api.yelp.com/v3/businesses/{}/reviews"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

# Search Parameters for Yelp API
params = {
    "term": "restaurants",
    "location": "Berkeley",
    "limit": 10,  # Number of businesses to retrieve
    "sort_by": "best_match"
}

response = requests.get(url_business, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Extract the list of businesses
    businesses = data.get("businesses", [])

    # Create an empty list to store business data with reviews
    business_data = []

    # Loop through each business and get the reviews
    for biz in businesses:
        # Extract business details
        business_info = {
            "Name": biz["name"],
            "ID": biz["id"],
            "Location": ", ".join(biz["location"]["display_address"]),
            "Rating": biz["rating"],
            "Review Count": biz["review_count"],
            "Phone": biz["phone"],
            "Price": biz.get("price", None),
            "Categories": ", ".join([cat["title"] for cat in biz["categories"]])
        }

        # Send request to get reviews for the business
        review_response = requests.get(url_reviews.format(biz["id"]), headers=headers)

        # Check if the review request was successful
        if review_response.status_code == 200:
            review_data = review_response.json().get("reviews", [])

            # Add reviews to business_info
            business_info["Reviews"] = "; ".join([rev["text"] for rev in review_data])

        # Append the business information (with reviews) to the list
        business_data.append(business_info)

        # Add a delay to avoid hitting the API rate limit
        time.sleep(0.1)

    # Create a DataFrame from the combined business and review data
    df = pd.DataFrame(business_data)

    # Display the DataFrame with reviews
else:
    print(f"Failed to retrieve businesses: {response.status_code}")


df.head(10)
# df.to_csv('my_dataframe.csv', index=False)

# from google.colab import files
# files.download('my_dataframe.csv')


,Name,ID,Location,Rating,Review Count,Phone,Price,Categories,Reviews
0,Kiraku,vZpuRwoCD4upNRP5CScjDQ,"2566B Telegraph Ave, Berkeley, CA 94704",4.4,1625,+15108482758,$$,"Tapas/Small Plates, Izakaya",Yummy!! Would definitely get the pasta & tosta...
1,Eureka!,XNZSLrX7UTU1G6TlaQZO0g,"2068 Center St, Berkeley, CA 94704",3.8,2253,+15108098282,$$,American,Went here for date night and really enjoyed th...
2,The Walnut Tavern Bar & Grill,Qx6aveiY4NH4hO2CfIeP6A,"1508 Walnut St, Berkeley, CA 94709",4.5,70,+14159666277,None,New American,The new owners did a nice job with the menu an...
3,Wood Tavern,bhnKl105GwMVVlsiUnwr2w,"6317 College Ave, Oakland, CA 94618",4.3,2284,+15106546607,$$,New American,I personally enjoyed the restaurant's signatur...
4,Creekwood,pJdwr6JoCVXTKbdJlm-rdg,"3121 Sacramento St, Berkeley, CA 94702",4.6,512,+15106478210,$$,"Italian, New American",Ambiance is nice... calm and relaxed. Service...
5,Farmhouse Kitchen Thai Cuisine,YEVnW6gJ9Hde6CYHF_u8iw,"1549 Shattuck Ave, Berkeley, CA 94709",4.7,283,+15106478024,None,"Thai, Cocktail Bars, Wine Bars",Really amazing food and service!! I strongly r...
6,Kuboba Spot,Qu6LoTMYDZNNNAPxyEBjog,"2618 Telegraph Ave, Berkeley, CA 94704",4.8,222,+15105294046,$,"Bubble Tea, Empanadas, Filipino",Why haven't I written a review yet!!? We love ...
7,Berkeley Social Club,oqyeLUSC2Z3Aczzh_4-4Yw,"2050 University Ave, Berkeley, CA 94704",4.1,2287,+15109005858,$$,"Breakfast & Brunch, Korean, Bars","The food, service, and the ambience are all wo..."
8,Angeline's Louisiana Kitchen,Q4o6ySw2V6r0UbK-aCAfng,"2261 Shattuck Ave, Berkeley, CA 94704",4.1,5019,+15105486900,$$,"Cajun/Creole, Southern, Seafood","The food was tasteless, the plating was messy,..."
9,Imm Thai Street Food,JTV1h_xD_3ZFzFomGWnp-w,"2068 University Ave, Berkeley, CA 94704",4.1,1936,+15108981123,$$,Thai,As someone who's tried upwards of 100 Pad See ...
